In [1]:
import os
import pandas as pd
import shutil

In [12]:
def list_dirs(directory):
    # 指定したディレクトリ内のすべてのサブフォルダをリスト化する
    dev_folders = [folder for folder in os.listdir(directory) if os.path.isdir(os.path.join(directory, folder))]
    return dev_folders

# フォルダのパスを指定
rootdir = r"C:\Users\r-fujita\Desktop\Phase2\教師データ"
if os.path.exists(rootdir):
    dev_dirs_list = list_dirs(rootdir)

# save_root_dirにC:\Users\r-fujita\Desktop\ET_Datasetを代入する
save_root_dir = r"C:\Users\r-fujita\Desktop\ET_Dataset"

# dev_dirs_listについてループ処理
for dev_dir in dev_dirs_list:
    # 参照先のパスを生成
    cd_dir_path = os.path.join(rootdir, dev_dir)
    
    # save_dirのパスを作成する
    save_dir_name = os.path.basename(cd_dir_path)
    save_dir = os.path.join(save_root_dir, save_dir_name)
    
    # 以下の処理はそのまま続ける
    # .d0と部分一致するファイルのパスを取得
    def find_d0_files(directory):
        d0_files = []
        for root, dirs, files in os.walk(directory):
            for file in files:
                if ".d0" in file:
                    d0_files.append(os.path.join(root, file))
        return d0_files

    # .d0と部分一致するファイルのパスを取得
    d0_files = find_d0_files(cd_dir_path)

    # 取得したパスをDataFrameに格納
    all_filepath_df = pd.DataFrame({'filepath': d0_files})

    # HHK.csvファイルを格納するためのリスト
    hhk_dataframes = []

    
    # 再帰的にフォルダを探索する関数
    def search_hhk_csv(folder_path):
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if file.startswith("HHK") and file.endswith(".csv"):
                    filepath = os.path.join(root, file)
                
                    # HHKファイルを読み込む
                    try:
                        # ファイル全体を読み込む
                        df = pd.read_csv(filepath, usecols=[0, 1, 6], encoding='shift_jis', header=None)
                    except pd.errors.EmptyDataError:
                        print(f"警告: ファイル '{file}' が空です。スキップします。")
                        continue
                        
                    except pd.errors.ParserError:
                        print(f"警告: ファイル '{filepath}' に6列目が存在しません。スキップします。")
                        continue

                    # DataFrameが空でない場合にのみ列を抽出
                    if not df.empty:
                            hhk_dataframes.append(df)

    # 親フォルダから再帰的にHHK.csvファイルを探索
    search_hhk_csv(cd_dir_path)

    # HHKファイルが存在しない場合は処理を終了する
    if not hhk_dataframes:
        print("HHKファイルに記録が見つかりませんでした。正常データをコピーして処理を終了します。")

        # save_dirを作成する
        os.makedirs(save_dir, exist_ok=True)
        
        # '正常'というフォルダを作成して、その中にall_filepath_dfの'filepath'と一致するファイルを全て保存する
        normal_dir = os.path.join(save_dir, '正常')
        os.makedirs(normal_dir, exist_ok=True)

        # all_filepath_dfのファイルを正常フォルダにコピーする
        for index, row in all_filepath_df.iterrows():
            filepath = row['filepath']
            filename = os.path.basename(filepath)
            shutil.copy(filepath, os.path.join(normal_dir, filename))

        continue 

    # 読み込んだすべてのHHK.csvファイルを連結
    if hhk_dataframes:  # hhk_dataframesが空でない場合のみ実行する
        combined_df = pd.concat(hhk_dataframes, ignore_index=True)
        # NaNを含む行を削除
        combined_df = combined_df.dropna()
        combined_df.columns = ['add1', 'add2', 'label']
    else:
        combined_df = pd.DataFrame()  # 空のDataFrameを作成する

    # 'add1'と'add2'の組み合わせで重複する行を特定
    duplicate_rows = combined_df[combined_df.duplicated(subset=['add1', 'add2'], keep=False)]

    # 重複行のうち、最初の行を残して削除
    combined_df = combined_df.drop_duplicates(subset=['add1', 'add2'], keep='first')

    # 'add1' と 'add2' をそれぞれ右側を0埋めして3桁にする
    combined_df['filename'] = combined_df['add1'].astype(str).str.zfill(3) + '-' + combined_df['add2'].astype(str).str.zfill(3)

    # ファイルパスを取得する関数
    def find_file_path(filename, directory):
        for root, dirs, files in os.walk(directory):
            for file in files:
                if filename + ".d" in file:
                    filepath = os.path.join(root, file)
                    # 最後の1文字を削除
                    filepath = filepath[:-1]
                    return filepath
        return None

    # "filename" 列の値を使用して "filepath" 列を作成
    combined_df['filepath'] = combined_df['filename'].apply(lambda x: find_file_path(x, cd_dir_path))


    # labelの種類を取得
    unique_labels = combined_df['label'].unique()

    # save_dirを作成する
    os.makedirs(save_dir, exist_ok=True)

    # unique_labelsと同じ名前のフォルダをsave_dir内に作成する
    for label in unique_labels:
        label_dir = os.path.join(save_dir, label)
        os.makedirs(label_dir, exist_ok=True)

    # 保存したファイルのパスを格納するリストを作成する
    saved_filepaths = []

    # combined_dfの各行に対して、該当するファイルを保存する
    for index, row in combined_df.iterrows():
        filename = row['filename'] + ".d0"  # combined_dfのファイル名
        label = row['label']
        save_path = os.path.join(save_dir, label)
        
        # cd_dir_pathを再帰的に検索して、filenameと部分一致するファイルを全てsave_pathに保存する
        for root, dirs, files in os.walk(cd_dir_path):
            for file in files:
                if filename in file:
                    filepath = os.path.join(root, file)
                    shutil.copy(filepath, save_path)
                    saved_filepaths.append(filepath)

    # 保存したファイルのパスをデータフレームに変換する
    saved_filepaths_df = pd.DataFrame(saved_filepaths, columns=['saved_filepath'])
    # all_filepath_dfからsaved_filepaths_dfと重複するファイルパスを削除する
    normal_filepath_df = all_filepath_df[~all_filepath_df['filepath'].isin(saved_filepaths_df['saved_filepath'])].copy()

    # '正常'というフォルダを作成して、その中にnormal_filepath_dfの'filepath'と一致するファイルを全て保存する
    normal_dir = os.path.join(save_dir, '正常')
    os.makedirs(normal_dir, exist_ok=True)

    # normal_filepath_dfのファイルを正常フォルダにコピーする
    for index, row in normal_filepath_df.iterrows():
        filepath = row['filepath']
        filename = os.path.basename(filepath)
        shutil.copy(filepath, os.path.join(normal_dir, filename))

警告: ファイル 'HHK.csv' が空です。スキップします。
HHKファイルに記録が見つかりませんでした。正常データをコピーして処理を終了します。
警告: ファイル 'HHK.csv' が空です。スキップします。
HHKファイルに記録が見つかりませんでした。正常データをコピーして処理を終了します。
警告: ファイル 'HHK.csv' が空です。スキップします。
警告: ファイル 'HHK.csv' が空です。スキップします。
HHKファイルに記録が見つかりませんでした。正常データをコピーして処理を終了します。
HHKファイルに記録が見つかりませんでした。正常データをコピーして処理を終了します。
HHKファイルに記録が見つかりませんでした。正常データをコピーして処理を終了します。
HHKファイルに記録が見つかりませんでした。正常データをコピーして処理を終了します。
警告: ファイル 'HHK.csv' が空です。スキップします。
HHKファイルに記録が見つかりませんでした。正常データをコピーして処理を終了します。
HHKファイルに記録が見つかりませんでした。正常データをコピーして処理を終了します。
警告: ファイル 'HHK.csv' が空です。スキップします。
HHKファイルに記録が見つかりませんでした。正常データをコピーして処理を終了します。
